# Preprocess subject data

In [1]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 


imported src module
imported preprocessing module
imported configs module
imported utils module


Load in subject IDs and paths from CFGLog

In [2]:
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC
Panc_subject_ids = config.data.chronic_pancreatitis.subject_ids.CP
Lupus_CP_subject_ids = config.data.lupus.subject_ids.CP
Lupus_NP_subject_ids = config.data.lupus.subject_ids.NP

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

Define preprocessing parameters

In [3]:
PERISTIM_TIME_WIN = 5

In [4]:
times_tup, time_win_path = pre_utils.get_time_window(PERISTIM_TIME_WIN)
time_range = (times_tup[0], times_tup[-1])

[-2.5,0.0,2.5]


Combine subject IDs

In [5]:
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids,
    "chronic_pancreatitis": Panc_subject_ids,
    "lupus": Lupus_CP_subject_ids + Lupus_NP_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain']) + len(sub_ids['chronic_pancreatitis']) + len(sub_ids['lupus'])}")
print(sub_ids)

Total subjects: 64
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.'], 'chronic_pancreatitis': ['002', '003', '006', '007', '008', '009', '010', '011', '012', '013', '014'], 'lupus': ['5186', '6310', '5295', '5873', '6100', '6106', '5648', '5675', '5845', '5713']}


Create Subject object for each subject

In [6]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        preprocessed_data_path = clbp_processed_path if group == "chronic_low_back_pain" else None
        subject = Subject(sub_id, group, preprocessed_data_path=preprocessed_data_path)
        print(subject.subject_folder)
        
        subject.preprocess()
        subject.get_cleaned_epochs(time_range, PERISTIM_TIME_WIN)           
        break

        # subject.get_cleaned_resting()

        # subject.get_stc_resting()
        # subject.get_stc_epochs()


018.EEGDATA
Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Chronic Low Back Pain Study/Data/Raw/018.EEGDATA/Acquisition 018.Part2 Data.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2959599  =      0.000 ...  2959.599 secs...


In [ ]:
subject.preprocessed_raw